<a href="https://colab.research.google.com/github/KBVKarthik/Google-Colaboratory-Notebooks/blob/main/Topic_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!wget https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_unprep.txt
!wget https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_prep.txt

In [2]:
%%capture
!pip install contextualized-topic-models==1.8.1
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [3]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import bert_embeddings_from_file, TopicModelDataPreparation
from contextualized_topic_models.datasets.dataset import CTMDataset
from contextualized_topic_models.evaluation.measures import CoherenceNPMI, InvertedRBO
from gensim.corpora.dictionary import Dictionary
from gensim.models import ldamodel 
import os
import numpy as np
import pickle

In [4]:
with open("dbpedia_sample_abstract_20k_prep.txt", 'r') as fr_prep:
  text_training_preprocessed = [line.strip() for line in fr_prep.readlines()]

with open("dbpedia_sample_abstract_20k_unprep.txt", 'r') as fr_unprep:
  text_training_not_preprocessed = [line.strip() for line in fr_unprep.readlines()]

In [5]:
assert len(text_training_preprocessed) == len(text_training_not_preprocessed)

print(text_training_not_preprocessed[0])
print(text_training_preprocessed[0])

The Mid-Peninsula Highway is a proposed freeway across the Niagara Peninsula in the Canadian province of Ontario. Although plans for a highway connecting Hamilton to Fort Erie south of the Niagara Escarpment have surfaced for decades,it was not until The Niagara Frontier International Gateway Study was published by the Ministry
mid peninsula highway proposed across peninsula canadian province ontario although highway connecting hamilton fort south international study published ministry


In [6]:
training_bow_documents = text_training_preprocessed[0:15000]
training_contextual_document = text_training_not_preprocessed[0:15000]

testing_bow_documents = text_training_preprocessed[15000:]
testing_contextual_documents = text_training_not_preprocessed[15000:]

In [7]:
tp = TopicModelDataPreparation("bert-base-nli-mean-tokens")

training_dataset = tp.create_training_set(training_contextual_document, training_bow_documents)

100%|██████████| 405M/405M [00:25<00:00, 15.8MB/s]
Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/sbert.net_models_bert-base-nli-mean-tokens/0_BERT were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Batches:   0%|          | 0/75 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [8]:
tp.vocab[:10]

['abbreviated',
 'academic',
 'academy',
 'access',
 'according',
 'achieved',
 'acquired',
 'acre',
 'acres',
 'across']

In [9]:
ctm = CombinedTM(input_size=len(tp.vocab), bert_input_size=768, num_epochs=100, n_components=50)
ctm.fit(training_dataset)  

Epoch: [100/100]	 Seen Samples: [1500000/1500000]	Train Loss: 134.98690813802082	Time: 0:00:07.259403: : 100it [12:20,  7.40s/it]


In [10]:
ctm.save(models_dir="./")

/usr/local/lib/python3.7/dist-packages/contextualized_topic_models/models/ctm.py:388: Warning: This is an experimental feature that we has not been fully tested. Refer to the following issue:https://github.com/MilaNLProc/contextualized-topic-models/issues/38
  Warning)


In [11]:
del ctm

In [15]:
ctm = CombinedTM(input_size=len(tp.vocab), bert_input_size=768, num_epochs=100, n_components=50)

ctm.load("/content/contextualized_topic_model_nc_50_tpm_0.0_tpv_0.98_hs_prodLDA_ac_(100, 100)_do_softplus_lr_0.2_mo_0.002_rp_0.99/",
                                                                                                      epoch=99)

/usr/local/lib/python3.7/dist-packages/contextualized_topic_models/models/ctm.py:413: Warning: This is an experimental feature that we has not been fully tested. Refer to the following issue:https://github.com/MilaNLProc/contextualized-topic-models/issues/38
  Warning)


In [16]:
ctm.get_topic_lists(5)

[['club', 'league', 'association', 'season', 'football'],
 ['chemical', 'thus', 'gene', 'star', 'away'],
 ['list', 'many', 'languages', 'language', 'word'],
 ['film', 'festival', 'directed', 'indian', 'award'],
 ['album', 'released', 'band', 'studio', 'rock'],
 ['greek', 'century', 'king', 'bc', 'de'],
 ['province', 'iran', 'population', 'central', 'district'],
 ['west', 'approximately', 'within', 'south', 'lies'],
 ['east', 'west', 'within', 'north', 'south'],
 ['son', 'became', 'john', 'british', 'father'],
 ['held', 'women', 'competition', 'figure', 'world'],
 ['best', 'known', 'actress', 'american', 'actor'],
 ['film', 'directed', 'stars', 'written', 'drama'],
 ['family', 'brown', 'mm', 'found', 'costa'],
 ['states', 'air', 'airport', 'united', 'list'],
 ['war', 'ii', 'world', 'class', 'army'],
 ['community', 'state', 'park', 'river', 'county'],
 ['member', 'liberal', 'conservative', 'council', 'parliament'],
 ['island', 'point', 'land', 'named', 'bay'],
 ['head', 'team', 'division

In [17]:
testing_dataset = tp.create_test_set(testing_contextual_documents, testing_bow_documents) 
predictions = ctm.get_doc_topic_distribution(testing_dataset, n_samples=10)

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/sbert.net_models_bert-base-nli-mean-tokens/0_BERT were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Batches:   0%|          | 0/25 [00:00<?, ?it/s]

Sampling: [10/10]: : 10it [00:20,  2.03s/it]


In [18]:
print(testing_contextual_documents[15])

topic_index = np.argmax(predictions[15])
ctm.get_topic_lists(5)[topic_index]

Dhale (Arabic: الضالع‎‎ Aḍ Ḍāliʿ) province is one of the governorates of Yemen that have been created after the announcement of Yemeni unification. The population of the province accounted for (2.4%) of the total population of the Republic, and allocated administratively into (9) districts. Dali city is the centre of


['province', 'iran', 'population', 'central', 'district']